In [ ]:
#### fumction that uploads the HARP dataset from input_root in local machone and creates train and test data
## Train and test files are saved to output root
## train, test - initialized dataframes. 
## partitian is made with 70% train in 30% test
#ind_test, ind_train - in case of partial run, can be used to save running time.
## Feature Extraction
def load_file(filepath):
    column_names = ['Ax','Ay','Az']
    dataframe = read_csv(filepath, header=None, names=column_names, delim_whitespace=True)
#     get_num_from_path = re.findall("\d",filepath)
    return dataframe


def load_dataset(input_root,train,test,ind_train,ind_test):
    files_list = glob.glob(input_root+'acc*.*')
    column_names = ['exp_id','user_id','act_id','label_start','label_end']
    labels = read_csv(input_root+'labels.txt', header=None, names=column_names, delim_whitespace=True)
    X = labels[['exp_id','user_id','label_start','label_end']] 
    Y = labels['act_id']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    #create_train_set
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    train = pd.DataFrame()
    for i in np.arange(ind_train,len(X_train)):
        #Validatin lenght of the segment so i will not be less then 2.56sec.
        #Otherwise the segment is excluded from the data
        if X_train.label_end[i]-X_train.label_start[i]>=129: 
            file_name='acc_exp{:0>2d}_user{:0>2d}.txt'.format(X_train.exp_id[i],X_train.user_id[i],y_train.act_id[i])
            train = train.append(
                featuer_extraction(input_root,file_name, X_train.label_start[i], X_train.label_end[i],y_train.act_id[i]),
                ignore_index=True)
        else:
            print('sample_'+str(i)+'was excluded')
    print('finish_train')

    #create_test_set
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    test = pd.DataFrame()
    for i in np.arange(ind_test,len(X_test)):
        #Validatin lenght of the segment so i will not be less then 2.56sec.
        #Otherwise the segment is excluded from the data
        if X_test.label_end[i]-X_test.label_start[i]>=129:
            file_name='acc_exp{:0>2d}_user{:0>2d}.txt'.format(X_test.exp_id[i],X_test.user_id[i],y_test.act_id[i])
            test = test.append(
                featuer_extraction(input_root,file_name, X_test.label_start[i], X_test.label_end[i],y_test.act_id[i]),
                ignore_index=True)
               
        else:
            print('sample_'+str(i)+'was excluded')
    print('finish_test')
    return train, test


## utility fimction to activate load_dataset function 
def extract_features_main(input_root,output_root):
    # insert ind_train = 780 and ind_test = 290 for a short run of the feature extraction module
    ind_train = 0
    ind_test = 0
    test = pd.DataFrame()
    train = pd.DataFrame()
    train,test = load_dataset(input_root,train,test,ind_train,ind_test)
    train.to_csv(output_root+'train_new.csv')
    test.to_csv(output_root+'test_new.csv')
    
#This code allow control on executing the model. 
#inpur root - where HARP database is saved in folder
input_root = r'C:/Users/Yair/Documents/Airtasker/Akhila/HAPT Data Set/RawData/'
#Output root - where to store fetures . using the same path to lunch them for in RandForest_model 
output_root = r'C:\Users\Yair\Documents\Airtasker\Akhila\HAPT Data Set\Extracted_Features\\'
### Deactivate this line to avoid feature extracture
extract_features_main(input_root,output_root)


